In [94]:
import os
import os

# Change to the correct directory where the EvoMan-Generalist folder is located
#os.chdir('/Users/christophlaute/Evolutionary/Group-Assignment-Evolutionary-Computing/EvoMan-Generalist')

# Verify the current working directory
#print("Current working directory:", os.getcwd())

#print("Current working directory:", os.getcwd())

import numpy as np
import random
from evoman.environment import Environment
from demo_controller import player_controller
import os
import copy
from scipy.stats import cauchy

from pymoo.indicators.hv import HV


In [3]:

n_hidden_neurons=10
#n_weights = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5
n_weights=265
experiment_name = 'generalist1'
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

In [7]:
def initialize_population(population_size, n_vars):
    population=[]
    individual_mutation_rates=[]
    for i in range(population_size):
        population.append(np.random.uniform(-1, 1, n_vars)) 
    return np.array(population)

pop=initialize_population(3,265) # for debugging

def simulation(ind,enemies=[2,4,6]): # just for debugging purpose set enemies as 2, 4, 6
    ''' simulates the environment and evaluates the fitness score against group of enemies
    returns fitness_score: list of fitness scores against each enemy and also average fitness too
    basically fitness_score[0]= fitness score upon enemies[0], fitness_score[1] = fitness score upon enemies[0]
    and fitness_score[3] = consolidated fitness( i.e mean(fitness_score) - standard deviation(fitness_score))'''

    individual_scores=[] 
    for i in range(len(enemies)):
      env = Environment(experiment_name=experiment_name,
                  enemies=[enemies[i]],
                  playermode="ai",
                  player_controller=player_controller(n_hidden_neurons),
                  enemymode="static",
                  level=2,
                  speed="fastest",
                  visuals=False)
      f,p,e,t = env.play(pcont=ind)
      individual_scores.append(f)
    
    #avg_fitness=env.cons_multi(np.array(individual_scores))
    #individual_scores.append(avg_fitness)

    return individual_scores


def evaluation(population, enemies:list):
   '''calls the simulation function for each individual in population 
      returns np array of fitness scores for whole population'''
   
   fitness_scores=[]
   for ind in population:
      individual_scores = simulation(ind,enemies)
      fitness_scores.append(individual_scores)

   return np.array(fitness_scores)
    
evaluation(pop,[2,4,6]) # debugging




      


MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.


array([[-5.90808294, 11.81585111,  3.15645558],
       [-5.79605775, -4.99043259, -5.17614973],
       [-5.768321  , -5.86929691, -5.30330491]])

In [8]:
def one_dimensional_hc(fitness_values,reference_point):
    """calculates the hypervolume in the case that only one dimenstion is left"""
    sort_fit_val= fitness_values[fitness_values.argsort()]
    hyper_volume=0
    hyper_volume = np.sum(reference_point - sort_fit_val) 
    return hyper_volume

def multi_dimensional_hc(fitness_values,reference_point):
    """recursively applies the hypervolume and computes the total hypervolume"""
    sort_fit_val= fitness_values[fitness_values[:,0].argsort()]
    hypervolume=0
    #computes hypervolume by adding contributions and multiplying by subvolumes
    for index,individual in enumerate(sort_fit_val[:,0]):
        cont = reference_point-individual
        fitness_copy = np.copy(sort_fit_val)
        new_fitness = np.delete(fitness_copy,0,axis = 1)
        sub_hyp = hypervolume_calculation(new_fitness[index:],reference_point)
        hypervolume += cont*sub_hyp
    return hypervolume

def hypervolume_calculation(fitness_values,reference_point):
    """applies the wfg algorithm to compute the hypervolume"""
    #sort individuals by fitness for first objective
    hypervolume=0
    #if only one dimension left
    if fitness_values.shape[1]==1:
        hypervolume=one_dimensional_hc(fitness_values,reference_point)
        return hypervolume

    #calculate contrubution to hypervolume for each individual
    hypervolume=multi_dimensional_hc(fitness_values,reference_point)
    return hypervolume


pop=initialize_population(10,265) # for debugging

fitness_values=evaluation(pop,[2,4,6])
reference_point=[0,0,0]
hv=hypervolume_calculation(fitness_values,0)
print(hv)
print(fitness_values.shape)
hv_calculator = HV(reference_point)
hv1 = hv_calculator.do(fitness_values)
print(hv1)
print(np.array_equal(hv,hv1)) # to check the correctness of code




        



MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: 

In [88]:
def hv_calculation(fitness_values,reference_point:list):
    hypervolume=0
    if len(fitness_values.shape)>1:
        if fitness_values.shape[1]==1:
            hypervolume =one_dimensional_hv(fitness_values,reference_point)
        else:
            hypervolume=multi_dimensional_hv(fitness_values,reference_point)
    return hypervolume

def one_dimensional_hv(fitness_values,reference_point:list):
    sorted_fitness=fitness_values[fitness_values.argsort()[::-1]] #for maximizing the objective function we need sort in descending order. to have that dominance effect
    hypervolume=0
    for current_point in sorted_fitness:
        if current_point[0]>reference_point[0]:
            hypervolume+=(current_point[0]-reference_point[0])
    return float(hypervolume)

def multi_dimensional_hv(fitness_values,reference_point:list):
    hypervolume=0
    sorted_first_fitness=fitness_values[fitness_values[:,0].argsort()[::-1]]
    for current_point in sorted_first_fitness:
        contribution=current_point[0]-reference_point[0]
        if fitness_values.shape[1] > 1: 
            new_fitness_values= fitness_values[:,1:] # removing first objective function
            new_reference_values= reference_point[1:] # removing reference point of first objective function too since its contribution is already considered
            hypervolume+=contribution*hv_calculation(new_fitness_values,new_reference_values)
    return float(hypervolume)

def hypervolume_survival_selection(population, fitness_values, reference_point, num_of_survivors):

    total_hypervolume=hv_calculation(fitness_values, reference_point)
    individual_contributions=[]
    for idx,individual in enumerate(population):
        new_fitness_values= np.delete(fitness_values,idx, axis=0)
        new_hypervolume=hv_calculation(new_fitness_values,reference_point)
        loss = total_hypervolume-new_hypervolume
        print(total_hypervolume,new_hypervolume)
        individual_contributions.append(loss)
    
    sort_individuals=population[np.array(individual_contributions).argsort()[::-1]]

    survivors=sort_individuals[:num_of_survivors]
    return survivors,individual_contributions
    
    








In [92]:
import numpy as np

# Your provided hypervolume-based survival selection function here

population = np.array(["agent1", "agent2", "agent3", "agent4", "agent5"])

fitness_values = np.array([
    [300, 200, 100],  # Fitness of agent1
    [310, 190, 110],  # Fitness of agent2
    [290, 210, 120],  # Fitness of agent3
    [320, 180, 130],  # Fitness of agent4
    [280, 220, 90]    # Fitness of agent5
])

reference_point = [250, 150, 80]
num_of_survivors = 3

# Run the hypervolume survival selection process
survivors = hypervolume_survival_selection(population, fitness_values, reference_point, num_of_survivors)
print("Selected survivors:", survivors)


6250000.0 4800000.0
6250000.0 3192000.0
6250000.0 3192000.0
6250000.0 3168000.0
6250000.0 3168000.0
Selected survivors: (array(['agent5', 'agent4', 'agent3'], dtype='<U6'), [1450000.0, 3058000.0, 3058000.0, 3082000.0, 3082000.0])


C:\Users\SAKTHE PRIYA\AppData\Local\Temp\ipykernel_33096\817826362.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(hypervolume)
